In [1]:
import numpy as np
from pyqpanda import CPUQVM, QProg,RY,RX,CZ,X, QCloud, real_chip_type, QPilotOSMachine
from pyqpanda import measure_all,convert_qprog_to_originir,finalize
import matplotlib.pyplot as plt
%matplotlib inline
from ErrorMitigation1 import data_read_txt, ResMatInv, ResMatInvN
from ErrorMitigation1 import data2prob, data2nprobs, PtrueCompute
from datetime import datetime
import time

In [2]:
def test_real_chip_query_task_statue(qm,task_id):
    res = 0
    while True:
        query_result = qm.query_task_state(task_id)
        if query_result[0] == '3':
            #print(f"state: {query_result[0]}")
            result = qm.parse_task_result(query_result[1])
            if len(result)<3:
                time.sleep(2)  # 延时2秒
                continue
            else:
                #print(f"state: {query_result[0]}")
                #print(f"result: {result}")
                res = result
                break
        elif query_result[0] == '4':
            #print(f"state: {query_result[0]}")
            print(f"Task measure failed Please measure later")
            res = -1
            break
        else:
            #print(f"state: {query_result[0]}")
            time.sleep(2) #延时2秒
            continue
    return res

def twoqubits(qid:list,q,c,bitstr='II'):
    # 输入线路参数生成线路.
    prog = QProg()
    if bitstr == 'IX':
        prog << X(q[qid[0]])
    elif bitstr == 'XI':
        prog << X(q[qid[1]])
    elif bitstr == 'XX':
        prog << X(q[qid[0]]) << X(q[qid[1]])
    else: 
        pass
    prog << measure_all([q[qid[0]],q[qid[1]]], [c[qid[0]],c[qid[1]]])
    return prog

def onequbit(qid:int, q, c, bitstr='I'):
    prog = QProg()
    if birstr == 'X':
        prog << X(q[qid])
    else:
        pass
    prog << measure_all(q[qid],c[qid])
    return prog

def CZprog(qid:int, q, c, bitstr='II'):
    # 输入线路参数生成线路.
    prog = QProg()
    prog << CZ(q[qid[0]], q[qid[1]])
    if bitstr == 'IX':
        prog << X(q[qid[0]])
    elif bitstr == 'XI':
        prog << X(q[qid[1]])
    elif bitstr == 'XX':
        prog << X(q[qid[0]]) << X(q[qid[1]])
    else: 
        pass
    prog << measure_all([q[qid[0]],q[qid[1]]], [c[qid[0]],c[qid[1]]])
    return prog

def pilot2qubits(qid:list, shots=10000,is_async=True, is_amend=True)->dict:
    qm = QPilotOSMachine('Pilot')
    qm.set_config(72,72)
    qm.init('https://10.10.10.61:10080',True,'ping','123456')
    #qm.set_qcloud_api('http://qcloud.originqc.com')
    
    q = qm.qAlloc_many(72)
    c = qm.cAlloc_many(72)
    bit_string = ['II','IX','XI','XX']
    
    res_c = []
    # calibration
    
    for bs in bit_string:
        # calibration
        prog1 = twoqubits(qid,q,c,bitstr=bs)
        if is_async:
            while True:
                task_id1 = qm.async_real_chip_measure(prog1, shots, chip_id=72, is_mapping=False, is_amend=is_amend)
                r_calib = test_real_chip_query_task_statue(qm,task_id1)
                if r_calib !=-1:
                    break
                else:
                    continue
        else:
            r_calib = qm.real_chip_measure(prog1, shots, chip_id=72, is_amend=is_amend,is_mapping=False)
        r_calib = dict2full(r_calib)
        r_dict1 = {'bitstr':bs,'shots':shots}
        r_dict1.update(r_calib)
        res_c.append(r_dict1)
        print('calib: ',list(r_dict1.values()))
        
    qm.finalize()
    return res_c

def pilot1qubit(qid:int, shots=10000,is_async=True, is_amend=True)->dict:
    qm = QPilotOSMachine('Pilot')
    qm.set_config(72,72)
    qm.init('https://10.10.10.61:10080',True,'ping','123456')
    #qm.set_qcloud_api('http://qcloud.originqc.com')
    
    q = qm.qAlloc_many(72)
    c = qm.cAlloc_many(72)
    bit_string = ['I','X']
    
    res_c = []
    # calibration
    
    for bs in bit_string:
        # calibration
        prog1 = onequibt(qid,q,c,bitstr=bs)
        if is_async:
            while True:
                task_id1 = qm.async_real_chip_measure(prog1, shots, chip_id=72, is_mapping=False, is_amend=is_amend)
                r_calib = test_real_chip_query_task_statue(qm,task_id1)
                if r_calib !=-1:
                    break
                else:
                    continue
        else:
            r_calib = qm.real_chip_measure(prog1, shots, chip_id=72, is_amend=is_amend,is_mapping=False)
        r_calib = dict2full(r_calib)
        r_dict1 = {'bitstr':bs,'shots':shots}
        r_dict1.update(r_calib)
        res_c.append(r_dict1)
        print('calib: ',list(r_dict1.values()))
        
    qm.finalize()
    return res_c

In [ ]:
from ErrorMitigation1 import filewrite
qubit_list = [45, 46, 52, 53, 54, 48]
shots = 10000
date = '0629_1'
root = "c:/Users/Origin31102/Desktop/real chip experiment/stping_test/dataAR/Calib_"
file1 = root+date+"_1qubit.txt"
file2 = root+date+"_2qubits.txt"


RES_2 = []
RES_1 = []
print("两比特的校准:")
for i in range(5):
    qid = [qubit_list[i], qubit_list[i+1]]
    print("qubit id: ", qid)
    res_c = pilot2qubits(qid, shots=10000,is_async=True, is_amend=True)
    RES_2.append(res_c)
    filewrite(file2, res_c, n)

print("单比特的校准:")
for i in range(6):
    qid =qubit_list[i]
    print("qubit id: ", qid)
    res_c = pilot1qubit(qid, shots=10000, is_async=True, is_amend=True)
    RES_1.append(res_c)
    filewrite(file1, res_c, n)